In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Check for missing values
print(df.isnull().sum())

# Convert 'TotalCharges' to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Drop rows with missing TotalCharges
df.dropna(subset=['TotalCharges'], inplace=True)

# Convert 'Churn' to binary
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Churn count
sns.countplot(data=df, x='Churn')
plt.title("Churn Count")

# Churn by contract type
sns.countplot(data=df, x='Contract', hue='Churn')
plt.title("Churn by Contract Type")

# Monthly Charges vs Churn
sns.boxplot(data=df, x='Churn', y='MonthlyCharges')
plt.title("Monthly Charges vs Churn")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Select relevant features
features = ['tenure', 'MonthlyCharges', 'TotalCharges']
X = df[features]
y = df['Churn']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
!pip install dash dash-bootstrap-components pandas plotly


In [ ]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

# Load dataset
df = pd.read_csv('Telco-Customer-Churn.csv')

# Data Cleaning
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(subset=['TotalCharges'], inplace=True)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Start Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

# Layout
app.layout = dbc.Container([
    html.H1("📉 Churn Analysis Dashboard", className="text-center my-4"),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Churn Rate"),
                dbc.CardBody(html.H4(f"{df['Churn'].mean()*100:.2f}%", className="card-title"))
            ], color="danger", inverse=True)
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Monthly Revenue Loss"),
                dbc.CardBody(html.H4(f"${(df[df['Churn'] == 1]['MonthlyCharges'].sum()):,.2f}", className="card-title"))
            ], color="warning", inverse=True)
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Average Tenure (Churned)"),
                dbc.CardBody(html.H4(f"{df[df['Churn'] == 1]['tenure'].mean():.1f} months", className="card-title"))
            ], color="info", inverse=True)
        ])
    ], className="mb-4"),

    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id="churn-by-contract",
                figure=px.histogram(df, x='Contract', color='Churn', barmode='group', title="Churn by Contract Type")
            )
        ])
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id="monthly-vs-churn",
                figure=px.box(df, x='Churn', y='MonthlyCharges', title="Monthly Charges vs Churn")
            )
        ]),
        dbc.Col([
            dcc.Graph(
                id="tenure-distribution",
                figure=px.histogram(df[df['Churn'] == 1], x='tenure', nbins=30, title="Tenure Distribution (Churned)")
            )
        ])
    ]),

    dbc.Row([
        dbc.Col([
            html.Label("Filter by Internet Service"),
            dcc.Dropdown(
                options=[{'label': i, 'value': i} for i in df['InternetService'].unique()],
                value='Fiber optic',
                id='internet-filter'
            ),
            dcc.Graph(id='churn-internet-service')
        ])
    ])
], fluid=True)

# Callback for filtered chart
@app.callback(
    Output('churn-internet-service', 'figure'),
    [Input('internet-filter', 'value')]
)
def update_chart(internet_choice):
    filtered = df[df['InternetService'] == internet_choice]
    fig = px.histogram(filtered, x='Contract', color='Churn', barmode='group',
                       title=f"Churn by Contract ({internet_choice})")
    return fig

# Run app in Jupyter
app.run_server(mode='inline', debug=False)
